In [1]:
!nvidia-smi

Wed Jun  1 19:37:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla V100-SXM2-16GB, 460.32.03, 16160 MiB


#Download dummy dataset, load csv and split to train val
Change data format to class folder stucture train & val

In [2]:
#Upload kaggle api key in /content/

! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! mkdir competitionKaggle
! pip install --upgrade --force-reinstall --no-deps kaggle

#https://www.kaggle.com/benjaminwarner/resized-2015-2019-blindness-detection-images
!kaggle datasets download -p /content/competitionKaggle/ -d benjaminwarner/resized-2015-2019-blindness-detection-images

!unzip /content/competitionKaggle/resized-2015-2019-blindness-detection-images.zip -d /content/competitionKaggle/
!rm /content/competitionKaggle/resized-2015-2019-blindness-detection-images.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 2.8 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=f479341f9f147fe5ba7964d55d8f1b01b3d8c3a38c6c031c922fa0c7dc73fe87
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
 19% 3.36G/17.3G [01:37<06:45, 37.0MB/s]
User cancelled operation
Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/util.py", line 300, in _run_finalizers
    finalizer()
  File "/usr/lib/python3.7/multiprocessing/util.py", line 224, in __call__
 

In [3]:
import pandas as pd

train = pd.read_csv("/content/competitionKaggle/labels/trainLabels15.csv")#"/content/competitionKaggle/trainLabels.csv")#"../input/com3025-2022-image-classification-challenge/train.csv"
train = train.astype('string')

test = pd.read_csv("/content/competitionKaggle/labels/testLabels15.csv")#"/content/competitionKaggle/trainLabels.csv")#"../input/com3025-2022-image-classification-challenge/train.csv"
test = test.astype('string')

train2 = pd.read_csv("/content/competitionKaggle/labels/trainLabels19.csv")#"/content/competitionKaggle/trainLabels.csv")#"../input/com3025-2022-image-classification-challenge/train.csv"
train2 = train2.astype('string')


def append_ext(fn):#add file extention to filenames
    return fn+".jpg"#".jpeg"#check converted file extention

train["image"]=train["image"].apply(append_ext)

test["image"]=test["image"].apply(append_ext)

train2["id_code"]=train2["id_code"].apply(append_ext)

#SAMPLE SMALLER FOR QUICK RUNS --------------------------------------------------------------------------------------------------------------------------------------------
#and making demo dataset
# train = train.sample(1000)
# test = test.sample(1000)
# train2 = train2.sample(1000)

FileNotFoundError: ignored

Convert data from directories with all images into train/{classnumber}/ folder structure

In [ ]:
!mkdir /content/competitionKaggle/train
!mkdir /content/competitionKaggle/val

In [ ]:
#very quick
import os
import shutil

def createFolderTreeFromDataframe(df, folder, source, ycolumn, imageCol):
  for index, row in df.iterrows():#for every row
    image_class = row[ycolumn]#load the class
    if not os.path.exists(f'/content/competitionKaggle/{folder}/{image_class}'):#create class folder if it doesnt exist
      os.makedirs(f'/content/competitionKaggle/{folder}/{image_class}')
    image_name = row[imageCol]#load the filename
    shutil.move(f"/content/competitionKaggle/{source}/{image_name}", f"/content/competitionKaggle/{folder}/{image_class}/")#move that file to the new dest

createFolderTreeFromDataframe(train, "train", "resized train 15",'level', 'image')
createFolderTreeFromDataframe(test, "train", "resized test 15",'level', 'image')#put test in train as well
createFolderTreeFromDataframe(train2, "train", "resized train 19", 'diagnosis', 'id_code')#put test in train as well
#92364 total images

# createFolderTreeFromDataframe(test, "val", "resized test 15")

If needed split into val with 20 of the images

In [ ]:
#----------------------put 20% of images in val
import os
import random
random.seed(0)

for i in range(5):
  files = os.listdir(f"/content/competitionKaggle/train/{i}")
  random.shuffle(files)
  files = files[:int(len(files)*0.2)]
  for fileName in files:
    if not os.path.exists(f'/content/competitionKaggle/val/{i}'):#create class folder if it doesnt exist
      os.makedirs(f'/content/competitionKaggle/val/{i}')
    shutil.move(f"/content/competitionKaggle/train/{i}/{fileName}", f"/content/competitionKaggle/val/{i}/")#move that file to the new dest

##make demo datset with val split or full train without train

In [ ]:
#move val and train to new dataset folder
# !mkdir /content/competitionKaggle/demodataset
# !mv /content/competitionKaggle/train /content/competitionKaggle/demodataset
# !mv /content/competitionKaggle/val /content/competitionKaggle/demodataset

In [ ]:
# #making demo dataset - delete after and not run again
# !kaggle datasets init -p /content/competitionKaggle/demodataset
# #change json metadata

In [ ]:
# !kaggle datasets create -p /content/competitionKaggle/demodataset/ --dir-mode zip

#Downlaod my kaggle full train DR - for unsupervised learning

In [4]:
#Upload kaggle api key in /content/

! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! mkdir competitionKaggle
! pip install --upgrade --force-reinstall --no-deps kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
mkdir: cannot create directory ‘competitionKaggle’: File exists
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle-1.5.12-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [5]:
!kaggle datasets download -p /content/competitionKaggle/ -d gilescodes/dr-full-train-set
!unzip /content/competitionKaggle/dr-full-train-set.zip -d /content/competitionKaggle/
!rm /content/competitionKaggle/dr-full-train-set.zip

Streaming output truncated to the last 5000 lines.
  inflating: /content/competitionKaggle/train/2/9744_left.jpg  
  inflating: /content/competitionKaggle/train/2/975252e325e3.jpg  
  inflating: /content/competitionKaggle/train/2/9752_left.jpg  
  inflating: /content/competitionKaggle/train/2/9759_right.jpg  
  inflating: /content/competitionKaggle/train/2/975_left.jpg  
  inflating: /content/competitionKaggle/train/2/975_right.jpg  
  inflating: /content/competitionKaggle/train/2/9761_left.jpg  
  inflating: /content/competitionKaggle/train/2/9764_left.jpg  
  inflating: /content/competitionKaggle/train/2/9764_right.jpg  
  inflating: /content/competitionKaggle/train/2/9780_left.jpg  
  inflating: /content/competitionKaggle/train/2/9783_left.jpg  
  inflating: /content/competitionKaggle/train/2/9785805af1b8.jpg  
  inflating: /content/competitionKaggle/train/2/9791_left.jpg  
  inflating: /content/competitionKaggle/train/2/9791_right.jpg  
  inflating: /content/competitionKaggle/train

#MoBY SLL SWIN T

## setup MOBY

In [6]:
!git clone https://github.com/SwinTransformer/Transformer-SSL.git
%cd /content/Transformer-SSL

# !pip3 install install pytorch==1.7.1 torchvision==0.8.2 cudatoolkit==10.1

#Experimenting with prepre-training---------------------------------------
# !git clone --single-branch --branch Modified-MoBY-Pretrained https://github.com/Giles-Billenness/Swin-Transformer.git
# %cd /content/Swin-Transformer

Cloning into 'Transformer-SSL'...
remote: Enumerating objects: 201, done.
remote: Total 201 (delta 0), reused 0 (delta 0), pack-reused 201
Receiving objects: 100% (201/201), 1.17 MiB | 5.12 MiB/s, done.
Resolving deltas: 100% (95/95), done.
/content/Transformer-SSL


In [7]:
#Install from VISSL notebook

#pytorch ssl certificate invalid :(
# !pip install --trusted-host pypi.org --trusted-host pytorch.org --trusted-host download.pytorch.org --trusted-host files.pypi.org --trusted-host files.pytorch.org --upgrade --force-reinstall torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# Install pytorch version 1.8
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install Apex by checking system settings: cuda version, pytorch version, and python version
import sys
import torch
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{torch.__version__[0:5:2]}"
])
print(version_str)

# install apex (pre-compiled with optimizer C++ extensions and CUDA kernels)
!pip install apex -f https://dl.fbaipublicfiles.com/vissl/packaging/apexwheels/{version_str}/download.html

import torch
print(torch.version.cuda)
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

!nvcc --version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 15 kB/s 
     |████████████████████████████████| 17.3 MB 748 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.8.0+cu101 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.0+cu101 which

In [8]:
%cd /content/Transformer-SSL
!pip install opencv-python==4.4.0.46 termcolor==1.1.0 yacs==0.1.8 diffdist

# !pip install timm==0.3.2
!pip install timm==0.4.9
!pip install PyYaml==5.1

/content/Transformer-SSL
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.5 MB 2.0 MB/s 
  Created wheel for diffdist: filename=diffdist-0.1-py3-none-any.whl size=6553 sha256=7959abf8a08d04ca7154d4e721e83ce4555fc1afc3a3ef99e66a6e85580a0619
  Stored in directory: /root/.cache/pip/wheels/58/eb/5d/4878c7186753c09afacb2a43a74b45289488ebc62e9576565e
Successfully built diffdist
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

## Run MoBY - change config to fit data first

In [9]:
%cd /content/Transformer-SSL

#or use --opts flag
batch_size = 64#64#32 #16 #8 #4 ish (grad overflows)
image_size = 224*1 #*2 #*3 #*4
TRAIN_EPOCHS = 300
SWIN_WINDOW_SIZE=7
# TRAIN_WARMUP_EPOCHS = 1
NUM_CLASSES = 5

SAVE_FREQ = 5#Save checkpoints every 5 epochs

!python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345  "/content/Transformer-SSL/moby_main.py" \
--cfg configs/moby_swin_tiny.yaml --data-path '/content/competitionKaggle' --batch-size $batch_size \
--output '/content/drive/MyDrive/Colab Notebooks/FYP/MOBY SSL SWIN/' \
--opts TRAIN.EPOCHS $TRAIN_EPOCHS MODEL.NUM_CLASSES $NUM_CLASSES DATA.IMG_SIZE $image_size \
SAVE_FREQ $SAVE_FREQ \
MODEL.SWIN.WINDOW_SIZE $SWIN_WINDOW_SIZE
# TRAIN.WARMUP_EPOCHS $TRAIN_WARMUP_EPOCHS
# --pretrained "/content/drive/MyDrive/Colab Notebooks/FYP/MOBY SSL SWIN/swin_tiny_patch4_window7_224.pth" \# doesnt work for moby

#Image Net pretrained models
#https://github.com/microsoft/Swin-Transformer
#https://github.com/SwinTransformer/Transformer-SSL
# #MoBy-PreTrained
# # /content/drive/MyDrive/Colab Notebooks/FYP/MOBY SSL SWIN/moby_swin_t_300ep_pretrained.pth
# #supervised
# #swin_tiny_patch4_window7_224.pth

/content/Transformer-SSL
=> merge config from configs/moby_swin_tiny.yaml
RANK and WORLD_SIZE in environ: 0/1
[2022-06-01 20:02:16 moby__swin_tiny__patch4_window7_224__odpr02_tdpr0_cm099_ct02_queue4096_proj2_pred2](moby_main.py 231): INFO Full config saved to /content/drive/MyDrive/Colab Notebooks/FYP/Final Code/moby__swin_tiny__patch4_window7_224__odpr02_tdpr0_cm099_ct02_queue4096_proj2_pred2/default/config.json
[2022-06-01 20:02:16 moby__swin_tiny__patch4_window7_224__odpr02_tdpr0_cm099_ct02_queue4096_proj2_pred2](moby_main.py 234): INFO AMP_OPT_LEVEL: O1
AUG:
  AUTO_AUGMENT: rand-m9-mstd0.5-inc1
  COLOR_JITTER: 0.4
  CUTMIX: 1.0
  CUTMIX_MINMAX: null
  MIXUP: 0.8
  MIXUP_MODE: batch
  MIXUP_PROB: 1.0
  MIXUP_SWITCH_PROB: 0.5
  RECOUNT: 1
  REMODE: pixel
  REPROB: 0.25
  SSL_AUG: true
  SSL_AUG_CROP: 0.08
  SSL_AUG_TYPE: byol
  SSL_LINEAR_AUG: false
BASE:
- ''
DATA:
  BATCH_SIZE: 64
  CACHE_MODE: part
  DATASET: imagenet
  DATA_PATH: /content/competitionKaggle
  IMG_SIZE: 224
  INTER

## Linear Evaluation
To evaluate a pre-trained MoBY with Swin Transformer on ImageNet-1K linear evaluation, run:

In [ ]:
# !python -m torch.distributed.launch --nproc_per_node <num-of-gpus-to-use> --master_port 12345 moby_linear.py \
# --cfg <config-file> --data-path <imagenet-path>

# # Notes:
# # Make sure the <config-file>, <output-directory> and <tag> are the same as in the pre-training stage.
# # Note that some configurations are fixed in moby_linear.py for simplicity.
# # For example, to evaluate MoBY Swin-T with 8 GPU on a single node on ImageNet-1K linear evluation, run:

# !python -m torch.distributed.launch --nproc_per_node 8 --master_port 12345  moby_linear.py \
# --cfg configs/moby_swin_tiny.yaml --data-path <imagenet-path> --batch-size 64

## Evaluate
To evaluate a MoBY with Swin Transformer linear evaluation model on ImageNet-1K, run:

In [ ]:
# # !python -m torch.distributed.launch --nproc_per_node <num-of-gpus-to-use> --master_port 12345 moby_linear.py \
# # --cfg <config-file> --resume <checkpoint> --data-path <imagenet-path> --eval

# # # For example, to evaluate the provided MoBY Swin-T linear evaluation model with a single GPU:
# %cd /content/Transformer-SSL

# batch_size = 32#64#32 #16 #8 #4 ish (grad overflows)
# image_size = 224*1 #*2 #*3 #*4
# START_EPOCH = 0
# TRAIN_EPOCHS = 300
# SWIN_WINDOW_SIZE=7
# # TRAIN_WARMUP_EPOCHS = 1
# NUM_CLASSES = 5
# OUTPUTPath = "/content/Transformer-SSL/output/"

# pretrainedModelPath = "/content/drive/MyDrive/Colab Notebooks/FYP/MOBY SSL SWIN/moby_swin_t_300ep_pretrained.pth"

# !python -m torch.distributed.launch --nproc_per_node 1 --master_port 12345 moby_linear.py \
# --cfg configs/moby_swin_tiny.yaml \
# --data-path "/content/competitionKaggle/" \
# --batch-size $batch_size \
# --opts MODEL.NUM_CLASSES $NUM_CLASSES DATA.IMG_SIZE $image_size TRAIN.EPOCHS $TRAIN_EPOCHS TRAIN.START_EPOCH $START_EPOCH \
# MODEL.SWIN.WINDOW_SIZE $SWIN_WINDOW_SIZE LINEAR_EVAL.PRETRAINED "/content/drive/MyDrive/Colab Notebooks/FYP/MOBY SSL SWIN/moby_swin_t_300ep_pretrained.pth"   --eval

# #comment out the overrides in linear parsing part for pretrained path
# #change "self.transform" to "[self.transform]" in /content/Transformer-SSL/data/custom_image_folder.py

# # --resume "/content/drive/MyDrive/Colab Notebooks/FYP/MOBY SSL SWIN/moby_swin_t_300ep_pretrained.pth" \
# # OUTPUT $OUTPUTPath

/content/Transformer-SSL
=> merge config from configs/moby_swin_tiny.yaml
RANK and WORLD_SIZE in environ: 0/1
[2022-04-22 00:39:38 moby__swin_tiny__patch4_window7_224__odpr02_tdpr0_cm099_ct02_queue4096_proj2_pred2](moby_linear.py 380): INFO Full config saved to output/moby__swin_tiny__patch4_window7_224__odpr02_tdpr0_cm099_ct02_queue4096_proj2_pred2/default/linear/config.json
[2022-04-22 00:39:38 moby__swin_tiny__patch4_window7_224__odpr02_tdpr0_cm099_ct02_queue4096_proj2_pred2](moby_linear.py 383): INFO AMP_OPT_LEVEL: O1
AUG:
  AUTO_AUGMENT: rand-m9-mstd0.5-inc1
  COLOR_JITTER: 0.4
  CUTMIX: 0.0
  CUTMIX_MINMAX: null
  MIXUP: 0.0
  MIXUP_MODE: batch
  MIXUP_PROB: 1.0
  MIXUP_SWITCH_PROB: 0.5
  RECOUNT: 1
  REMODE: pixel
  REPROB: 0.25
  SSL_AUG: false
  SSL_AUG_CROP: 0.08
  SSL_AUG_TYPE: byol
  SSL_LINEAR_AUG: true
BASE:
- ''
DATA:
  BATCH_SIZE: 32
  CACHE_MODE: part
  DATASET: imagenet
  DATA_PATH: /content/competitionKaggle/
  IMG_SIZE: 224
  INTERPOLATION: bicubic
  NUM_WORKERS: 8
